# TCGA Data Preprocessing
---

Preprocessing the TCGA dataset from the Pancancer paper (https://www.ncbi.nlm.nih.gov/pubmed/29625048).

The Cancer Genome Atlas (TCGA), a landmark cancer genomics program, molecularly characterized over 20,000 primary cancer and matched normal samples spanning 33 cancer types. This joint effort between the National Cancer Institute and the National Human Genome Research Institute began in 2006, bringing together researchers from diverse disciplines and multiple institutions.

## Importing the necessary packages

In [1]:
import os                                  # os handles directory/workspace changes
import yaml                                # Save and load YAML files
import numpy as np                         # NumPy to handle numeric and NaN operations
from functools import reduce               # Parallelize functions

In [2]:
import os
import pandas 

# Set base data path
data_path = '/Users/elenalickel/Desktop/Thesis/IRR_Code/TCGA-Pancancer/'

# File names
rna_file = 'EBPlusPlusAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv'
abs_file = 'TCGA_mastercalls.abs_segtabs.fixed.txt'
cdr_file = 'TCGA-CDR-SupplementalTableS1.xlsx'

# Full paths
rna_path = os.path.join(data_path, rna_file)
abs_path = os.path.join(data_path, abs_file)
cdr_path = os.path.join(data_path, cdr_file)

# Load RNA and transpose
rna_df = pandas.read_csv(rna_path, sep='\t')
rna_df = rna_df.set_index('gene_id').transpose()
print("RNA shape:", rna_df.shape)
display(rna_df.head())

# Load ABSOLUTE CNV data
abs_df = pandas.read_csv(abs_path, sep='\t')
print("ABSOLUTE shape:", abs_df.shape)
display(abs_df.head())

# Load Clinical Data Resource (CDR)
cdr_df = pandas.read_excel(cdr_path)
print("CDR shape:", cdr_df.shape)
display(cdr_df.head())

RNA shape: (11069, 20531)


gene_id,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,?|317712,...,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
TCGA-OR-A5J1-01A-11R-A29S-07,0.0,3.2661,3.9385,149.1350,2034.10,0.0,274.255,1.4409,0.0,0.0,...,183.9580,146.9740,20.6532,351.585,1050.910,0.4803,648.415,1841.020,1157.540,596.062
TCGA-OR-A5J2-01A-11R-A29S-07,0.0,2.6815,8.9948,81.0777,1304.93,0.0,199.302,0.0000,0.0,0.0,...,264.1250,438.4640,57.1735,279.023,2979.870,31.4052,1166.020,3059.990,1895.990,801.637
TCGA-OR-A5J3-01A-11R-A29S-07,0.0,1.7301,6.5650,86.4879,1054.66,0.0,348.393,0.5925,0.0,0.0,...,90.6532,190.1940,39.6978,495.334,914.827,0.5925,806.399,2655.610,1482.450,437.269
TCGA-OR-A5J5-01A-11R-A29S-07,0.0,0.0000,1.5492,53.9117,2350.89,0.0,439.194,0.7746,0.0,0.0,...,260.2630,840.4340,17.8156,206.042,890.782,11.6189,553.834,2367.930,1140.200,512.781
TCGA-OR-A5J6-01A-31R-A29S-07,0.0,0.0000,4.4709,66.9063,1257.99,0.0,149.215,0.0000,0.0,0.0,...,64.2685,63.1508,12.8537,117.919,894.730,7.8240,795.812,708.071,796.371,475.587


ABSOLUTE shape: (837726, 20)


,Sample,Chromosome,Start,End,Num_Probes,Length,Modal_HSCN_1,Modal_HSCN_2,Modal_Total_CN,Subclonal_HSCN_a1,Subclonal_HSCN_a2,Cancer_cell_frac_a1,Ccf_ci95_low_a1,Ccf_ci95_high_a1,Cancer_cell_frac_a2,Ccf_ci95_low_a2,Ccf_ci95_high_a2,LOH,Homozygous_deletion,solution
0,TCGA-OR-A5J1-01,1.0,564621.0,8644007.0,2257.0,8079386.0,1.0,1.0,2.0,0.0,0.0,0.05,0.02170,0.07127,0.05,0.02170,0.07127,0.0,0.0,new
1,TCGA-OR-A5J1-01,1.0,8652154.0,9140078.0,121.0,487924.0,1.0,2.0,3.0,0.0,0.0,0.08,0.00356,0.14847,0.95,0.86228,1.00000,0.0,0.0,new
2,TCGA-OR-A5J1-01,1.0,9143332.0,12837840.0,848.0,3694508.0,1.0,1.0,2.0,0.0,0.0,0.06,0.02493,0.08426,0.06,0.02493,0.08426,0.0,0.0,new
3,TCGA-OR-A5J1-01,1.0,12852748.0,16846035.0,1120.0,3993287.0,1.0,1.0,2.0,0.0,0.0,0.03,0.00000,0.04964,0.03,0.00000,0.04964,0.0,0.0,new
4,TCGA-OR-A5J1-01,1.0,17246171.0,25570830.0,2789.0,8324659.0,1.0,1.0,2.0,0.0,0.0,0.04,0.01486,0.06423,0.04,0.01476,0.06412,0.0,0.0,new


CDR shape: (11160, 34)


,Unnamed: 0,bcr_patient_barcode,type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,...,residual_tumor,OS,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction
0,1,TCGA-OR-A5J1,ACC,58.0,MALE,WHITE,Stage II,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],...,NaN,1.0,1355.0,1.0,1355.0,1.0,754.0,1.0,754.0,NaN
1,2,TCGA-OR-A5J2,ACC,44.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],...,NaN,1.0,1677.0,1.0,1677.0,NaN,NaN,1.0,289.0,NaN
2,3,TCGA-OR-A5J3,ACC,23.0,FEMALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],...,NaN,0.0,2091.0,0.0,2091.0,1.0,53.0,1.0,53.0,NaN
3,4,TCGA-OR-A5J4,ACC,23.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],...,NaN,1.0,423.0,1.0,423.0,NaN,NaN,1.0,126.0,NaN
4,5,TCGA-OR-A5J5,ACC,30.0,MALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],...,NaN,1.0,365.0,1.0,365.0,NaN,NaN,1.0,50.0,NaN


In [3]:
import modin.pandas as pd                  # Optimized distributed version of Pandas

Allow pandas to show more columns:

In [4]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

Set the random seed for reproducibility:

In [5]:
np.random.seed(42)

## RNA

### Normalizing data

In [6]:
rna_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
gene_id,,,,,,,,
?|100130426,9351.0,0.028386,0.239364,-0.090423,0.00000,0.000000,0.000000,17.995000
?|100133144,11069.0,11.251089,12.199371,-0.525753,3.44920,7.809006,14.976397,187.474945
?|100134869,11069.0,11.670693,11.671968,-0.516280,4.59750,8.640000,14.987820,402.218988
?|10357,11069.0,160.950078,126.505719,0.000000,85.28260,123.578000,191.223000,1995.021959
?|10431,11069.0,986.122273,416.277222,232.315000,705.77300,905.364000,1172.160000,6818.118303
...,...,...,...,...,...,...,...,...
ZYG11A|440590,11069.0,55.909279,173.232482,-0.402069,1.28640,11.078700,58.539600,9050.870000
ZYG11B|79699,11069.0,924.782456,480.222826,46.019981,634.85406,834.735000,1114.529951,21058.900000
ZYX|7791,11069.0,4669.085953,3148.075907,139.635000,2685.23000,3983.570000,5729.070000,49294.800000


The data is not (well) normalized yet. All columns should have 0 mean and 1 standard deviation.

Save the dataframe before normalizing:

In [7]:
import os

# Ensure the output directory exists
output_dir = os.path.join(data_path, 'cleaned/unnormalized')
os.makedirs(output_dir, exist_ok=True)

# Now save the file
rna_df.to_csv(os.path.join(output_dir, 'rna.csv'))

Normalize the data into a new dataframe:

In [8]:
from sklearn.preprocessing import StandardScaler

# Initialize scaler
scaler = StandardScaler()

# Fit and transform the RNA data
rna_df_norm = pandas.DataFrame(
    scaler.fit_transform(rna_df),
    index=rna_df.index,
    columns=rna_df.columns
)

rna_df_norm.head()


gene_id,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,?|317712,?|340602,?|388795,?|390284,?|391343,?|391714,?|404770,?|441362,?|442388,?|553137,?|57714,?|645851,?|652919,?|653553,?|728045,?|728603,?|728788,?|729884,?|8225,?|90288,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2M|2,A2ML1|144568,A4GALT|53947,A4GNT|51146,AAA1|404744,AAAS|8086,AACS|65985,AACSL|729522,AADAC|13,AADACL2|344752,AADACL3|126767,AADACL4|343066,AADAT|51166,AAGAB|79719,AAK1|22848,AAMP|14,AANAT|15,AARS|16,AARS2|57505,AARSD1|80755,AASDH|132949,AASDHPPT|60496,AASS|10157,AATF|26574,AATK|9625,ABAT|18,ABCA10|10349,ABCA1|19,ABCA11P|79963,ABCA12|26154,ABCA13|154664,ABCA17P|650655,ABCA2|20,ABCA3|21,ABCA4|24,ABCA5|23461,ABCA6|23460,ABCA7|10347,ABCA8|10351,ABCA9|10350,ABCB10|23456,ABCB11|8647,ABCB1|5243,ABCB4|5244,ABCB5|340273,ABCB6|10058,ABCB7|22,ABCB8|11194,ABCB9|23457,ABCC10|89845,ABCC11|85320,ABCC12|94160,ABCC13|150000,ABCC1|4363,ABCC2|1244,ABCC3|8714,ABCC4|10257,ABCC5|10057,ABCC6|368,ABCC6P1|653190,ABCC6P2|730013,ABCC8|6833,ABCC9|10060,ABCD1|215,ABCD2|225,ABCD3|5825,ABCD4|5826,ABCE1|6059,ABCF1|23,ABCF2|10061,ABCF3|55324,ABCG1|9619,ABCG2|9429,ABCG4|64137,ABCG5|64240,ABCG8|64241,ABHD10|55347,ABHD11|83451,ABHD12|26090,ABHD12B|145447,ABHD13|84945,ABHD14A|25864,ABHD14B|84836,ABHD15|116236,ABHD1|84696,ABHD2|11057,ABHD3|171586,ABHD4|63874,ABHD5|51099,ABHD6|57406,ABHD8|79575,ABI1|10006,ABI2|10152,ABI3|51225,ABI3BP|25890,ABL1|25,ABL2|27,ABLIM1|3983,ABLIM2|84448,ABLIM3|22885,ABO|28,ABP1|26,ABR|29,ABRA|137735,ABT1|29777,ABTB1|80325,ABTB2|25841,ACAA1|30,ACAA2|10449,ACACA|31,ACACB|32,ACAD10|80724,ACAD11|84129,ACAD8|27034,ACAD9|28976,ACADL|33,ACADM|34,ACADS|35,ACADSB|36,ACADVL|37,ACAN|176,ACAP1|9744,ACAP2|23527,ACAP3|116983,ACAT1|38,ACAT2|39,ACBD3|64746,ACBD4|79777,ACBD5|91452,ACBD6|84320,ACBD7|414149,ACCN1|40,ACCN2|41,ACCN3|9311,ACCN4|55515,ACCN5|51802,ACCS|84680,ACCSL|390110,ACD|65057,ACE|1636,ACE2|59272,ACER1|125981,ACER2|340485,ACER3|55331,ACHE|43,ACIN1|22985,ACLY|47,ACMSD|130013,ACN9|57001,ACO1|48,ACO2|50,ACOT11|26027,ACOT12|134526,ACOT13|55856,ACOT1|641371,ACOT2|10965,ACOT4|122970,ACOT6|641372,ACOT7|11332,ACOT8|10005,ACOT9|23597,ACOX1|51,ACOX2|8309,ACOX3|8310,ACOXL|55289,ACP1|52,ACP2|53,ACP5|54,ACP6|51205,ACPL2|92370,ACPP|55,ACPT|93650,ACR|49,ACRBP|84519,ACRC|93953,ACRV1|56,ACSBG1|23205,ACSBG2|81616,ACSF2|80221,ACSF3|197322,ACSL1|2180,ACSL3|2181,ACSL4|2182,ACSL5|51703,ACSL6|23305,ACSM1|116285,ACSM2A|123876,ACSM2B|348158,ACSM3|6296,ACSM4|341392,ACSM5|54988,ACSS1|84532,ACSS2|55902,ACSS3|79611,ACTA1|58,ACTA2|59,ACTB|60,ACTBL2|345651,ACTC1|70,ACTG1|71,ACTG2|72,ACTL6A|86,ACTL6B|51412,ACTL7A|10881,ACTL7B|10880,ACTL8|81569,ACTL9|284382,ACTN1|87,ACTN2|88,ACTN3|89,ACTN4|81,ACTR10|55860,ACTR1A|10121,ACTR1B|10120,ACTR2|10097,ACTR3|10096,ACTR3B|57180,ACTR3C|653857,ACTR5|79913,ACTR6|64431,ACTR8|93973,ACTRT1|139741,ACTRT2|140625,ACVR1|90,ACVR1B|91,ACVR1C|130399,ACVR2A|92,ACVR2B|93,ACVRL1|94,ACY1|95,ACY3|91703,ACYP1|97,ACYP2|98,ADA|100,ADAD1|132612,ADAD2|161931,ADAL|161823,ADAM10|102,ADAM11|4185,ADAM12|8038,ADAM15|8751,ADAM17|6868,ADAM18|8749,ADAM19|8728,ADAM20|8748,ADAM21|8747,ADAM21P1|145241,ADAM2|2515,ADAM22|53616,ADAM23|8745,ADAM28|10863,ADAM29|11086,ADAM30|11085,ADAM32|203102,ADAM33|80332,ADAM3A|1587,ADAM5P|255926,ADAM6|8755,ADAM7|8756,ADAM8|101,ADAM9|8754,ADAMDEC1|27299,ADAMTS10|81794,ADAMTS12|81792,ADAMTS13|11093,ADAMTS14|140766,ADAMTS15|170689,ADAMTS16|170690,ADAMTS17|170691,ADAMTS18|170692,ADAMTS19|171019,ADAMTS1|9510,ADAMTS20|80070,ADAMTS2|9509,ADAMTS3|9508,ADAMTS4|9507,ADAMTS5|11096,ADAMTS6|11174,ADAMTS7|11173,ADAMTS8|11095,ADAMTS9|56999,ADAMTSL1|92949,ADAMTSL2|9719,ADAMTSL3|57188,ADAMTSL4|54507,ADAMTSL5|339366,ADAP1|11033,ADAP2|55803,ADAR|103,ADARB1|104,ADARB2|105,ADAT1|23536,ADAT2|134637,ADAT3|113179,ADC|113451,ADCK1|57143,ADCK2|90956,ADCK4|79934,ADCK5|203054,ADCY10|55811,ADCY1|107,ADCY2|108,ADCY3|109,ADCY4|196883,ADCY5|111,ADCY6|112,ADCY7|113,ADCY8|114,ADCY9|115,ADCYAP1|116,ADCYAP1R1|117,ADD1|118,ADD2|119,ADD3|120,ADH1A|124,ADH1B|125,

Confirm that everything is ok through the `describe` method:

In [9]:
rna_df_norm.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
gene_id,,,,,,,,
?|100130426,9351.0,3.039430e-16,1.000053,-0.496382,-0.118597,-0.118597,-0.118597,75.063966
?|100133144,11069.0,-2.834725e-15,1.000045,-0.965408,-0.639561,-0.282165,0.305383,14.445976
?|100134869,11069.0,5.915947e-15,1.000045,-1.044170,-0.606026,-0.259667,0.284209,33.461876
?|10357,11069.0,-5.546200e-15,1.000045,-1.272333,-0.598162,-0.295431,0.239312,14.498591
?|10431,11069.0,-9.859912e-15,1.000045,-1.810912,-0.673498,-0.194010,0.446928,14.010518
...,...,...,...,...,...,...,...,...
ZYG11A|440590,11069.0,1.591965e-15,1.000045,-0.325077,-0.315330,-0.258800,0.015184,51.926561
ZYG11B|79699,11069.0,-9.757204e-16,1.000045,-1.829988,-0.603764,-0.187520,0.395142,41.928509
ZYX|7791,11069.0,1.797380e-15,1.000045,-1.438865,-0.630209,-0.217767,0.336724,14.176192


Save the normalized dataframe:

In [10]:
normalized_dir = os.path.join(data_path, 'cleaned/normalized')
os.makedirs(normalized_dir, exist_ok=True)

# Save the normalized RNA data
rna_df_norm.to_csv(os.path.join(normalized_dir, 'rna.csv'))


In [11]:
rna_df_norm.head()

gene_id,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,?|317712,?|340602,?|388795,?|390284,?|391343,?|391714,?|404770,?|441362,?|442388,?|553137,?|57714,?|645851,?|652919,?|653553,?|728045,?|728603,?|728788,?|729884,?|8225,?|90288,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2M|2,A2ML1|144568,A4GALT|53947,A4GNT|51146,AAA1|404744,AAAS|8086,AACS|65985,AACSL|729522,AADAC|13,AADACL2|344752,AADACL3|126767,AADACL4|343066,AADAT|51166,AAGAB|79719,AAK1|22848,AAMP|14,AANAT|15,AARS|16,AARS2|57505,AARSD1|80755,AASDH|132949,AASDHPPT|60496,AASS|10157,AATF|26574,AATK|9625,ABAT|18,ABCA10|10349,ABCA1|19,ABCA11P|79963,ABCA12|26154,ABCA13|154664,ABCA17P|650655,ABCA2|20,ABCA3|21,ABCA4|24,ABCA5|23461,ABCA6|23460,ABCA7|10347,ABCA8|10351,ABCA9|10350,ABCB10|23456,ABCB11|8647,ABCB1|5243,ABCB4|5244,ABCB5|340273,ABCB6|10058,ABCB7|22,ABCB8|11194,ABCB9|23457,ABCC10|89845,ABCC11|85320,ABCC12|94160,ABCC13|150000,ABCC1|4363,ABCC2|1244,ABCC3|8714,ABCC4|10257,ABCC5|10057,ABCC6|368,ABCC6P1|653190,ABCC6P2|730013,ABCC8|6833,ABCC9|10060,ABCD1|215,ABCD2|225,ABCD3|5825,ABCD4|5826,ABCE1|6059,ABCF1|23,ABCF2|10061,ABCF3|55324,ABCG1|9619,ABCG2|9429,ABCG4|64137,ABCG5|64240,ABCG8|64241,ABHD10|55347,ABHD11|83451,ABHD12|26090,ABHD12B|145447,ABHD13|84945,ABHD14A|25864,ABHD14B|84836,ABHD15|116236,ABHD1|84696,ABHD2|11057,ABHD3|171586,ABHD4|63874,ABHD5|51099,ABHD6|57406,ABHD8|79575,ABI1|10006,ABI2|10152,ABI3|51225,ABI3BP|25890,ABL1|25,ABL2|27,ABLIM1|3983,ABLIM2|84448,ABLIM3|22885,ABO|28,ABP1|26,ABR|29,ABRA|137735,ABT1|29777,ABTB1|80325,ABTB2|25841,ACAA1|30,ACAA2|10449,ACACA|31,ACACB|32,ACAD10|80724,ACAD11|84129,ACAD8|27034,ACAD9|28976,ACADL|33,ACADM|34,ACADS|35,ACADSB|36,ACADVL|37,ACAN|176,ACAP1|9744,ACAP2|23527,ACAP3|116983,ACAT1|38,ACAT2|39,ACBD3|64746,ACBD4|79777,ACBD5|91452,ACBD6|84320,ACBD7|414149,ACCN1|40,ACCN2|41,ACCN3|9311,ACCN4|55515,ACCN5|51802,ACCS|84680,ACCSL|390110,ACD|65057,ACE|1636,ACE2|59272,ACER1|125981,ACER2|340485,ACER3|55331,ACHE|43,ACIN1|22985,ACLY|47,ACMSD|130013,ACN9|57001,ACO1|48,ACO2|50,ACOT11|26027,ACOT12|134526,ACOT13|55856,ACOT1|641371,ACOT2|10965,ACOT4|122970,ACOT6|641372,ACOT7|11332,ACOT8|10005,ACOT9|23597,ACOX1|51,ACOX2|8309,ACOX3|8310,ACOXL|55289,ACP1|52,ACP2|53,ACP5|54,ACP6|51205,ACPL2|92370,ACPP|55,ACPT|93650,ACR|49,ACRBP|84519,ACRC|93953,ACRV1|56,ACSBG1|23205,ACSBG2|81616,ACSF2|80221,ACSF3|197322,ACSL1|2180,ACSL3|2181,ACSL4|2182,ACSL5|51703,ACSL6|23305,ACSM1|116285,ACSM2A|123876,ACSM2B|348158,ACSM3|6296,ACSM4|341392,ACSM5|54988,ACSS1|84532,ACSS2|55902,ACSS3|79611,ACTA1|58,ACTA2|59,ACTB|60,ACTBL2|345651,ACTC1|70,ACTG1|71,ACTG2|72,ACTL6A|86,ACTL6B|51412,ACTL7A|10881,ACTL7B|10880,ACTL8|81569,ACTL9|284382,ACTN1|87,ACTN2|88,ACTN3|89,ACTN4|81,ACTR10|55860,ACTR1A|10121,ACTR1B|10120,ACTR2|10097,ACTR3|10096,ACTR3B|57180,ACTR3C|653857,ACTR5|79913,ACTR6|64431,ACTR8|93973,ACTRT1|139741,ACTRT2|140625,ACVR1|90,ACVR1B|91,ACVR1C|130399,ACVR2A|92,ACVR2B|93,ACVRL1|94,ACY1|95,ACY3|91703,ACYP1|97,ACYP2|98,ADA|100,ADAD1|132612,ADAD2|161931,ADAL|161823,ADAM10|102,ADAM11|4185,ADAM12|8038,ADAM15|8751,ADAM17|6868,ADAM18|8749,ADAM19|8728,ADAM20|8748,ADAM21|8747,ADAM21P1|145241,ADAM2|2515,ADAM22|53616,ADAM23|8745,ADAM28|10863,ADAM29|11086,ADAM30|11085,ADAM32|203102,ADAM33|80332,ADAM3A|1587,ADAM5P|255926,ADAM6|8755,ADAM7|8756,ADAM8|101,ADAM9|8754,ADAMDEC1|27299,ADAMTS10|81794,ADAMTS12|81792,ADAMTS13|11093,ADAMTS14|140766,ADAMTS15|170689,ADAMTS16|170690,ADAMTS17|170691,ADAMTS18|170692,ADAMTS19|171019,ADAMTS1|9510,ADAMTS20|80070,ADAMTS2|9509,ADAMTS3|9508,ADAMTS4|9507,ADAMTS5|11096,ADAMTS6|11174,ADAMTS7|11173,ADAMTS8|11095,ADAMTS9|56999,ADAMTSL1|92949,ADAMTSL2|9719,ADAMTSL3|57188,ADAMTSL4|54507,ADAMTSL5|339366,ADAP1|11033,ADAP2|55803,ADAR|103,ADARB1|104,ADARB2|105,ADAT1|23536,ADAT2|134637,ADAT3|113179,ADC|113451,ADCK1|57143,ADCK2|90956,ADCK4|79934,ADCK5|203054,ADCY10|55811,ADCY1|107,ADCY2|108,ADCY3|109,ADCY4|196883,ADCY5|111,ADCY6|112,ADCY7|113,ADCY8|114,ADCY9|115,ADCYAP1|116,ADCYAP1R1|117,ADD1|118,ADD2|119,ADD3|120,ADH1A|124,ADH1B|125,

## ABSOLUTE-annotated seg data

This dataframe contains copy-number and copy-ratio related data.

Copy number alterations/aberrations (CNAs) are changes in copy number that have arisen in somatic tissue (for example, just in a tumor), copy number variations (CNVs) originated from changes in copy number in germline cells (and are thus in all cells of the organism).

The rows correspond to contiguous chunks along the chromosome with the same DNA copy-number. "Chromosome" is the chromosome, can be 1-22, X or Y (see human genome). Start is the physical start location for the segment along said linear chromosome, end is the end coordinate. Num_probes is the number of SNP-array probes falling within the segment (these were used to call copy numbers). Reference: https://www.biostars.org/p/244374/

### Loading the data

In [12]:
abs_anttd_seg_df = abs_df

In [13]:
len(abs_anttd_seg_df)

837726

### Converting categorical features to numeric

In [14]:
abs_anttd_seg_df.solution.value_counts()

solution
new    837725
Name: count, dtype: int64

In [15]:
abs_anttd_seg_df.solution = abs_anttd_seg_df.solution.apply(lambda x: 1 if x == 'new' else 0)
abs_anttd_seg_df = abs_anttd_seg_df.rename(columns={'solution': 'new_solution'})
abs_anttd_seg_df.new_solution.value_counts()

new_solution
1    837725
0         1
Name: count, dtype: int64

### Removing unneeded features

Columns `Start`, `End`, `Num_Probes` and `Length` don't seem to be relevant as we don't need to know so much detail of each chromosome nor experiment specific information.

In [16]:
abs_anttd_seg_df = abs_anttd_seg_df.drop(columns=['Start', 'End', 'Num_Probes', 'Length'], axis=1)
abs_anttd_seg_df.head()

,Sample,Chromosome,Modal_HSCN_1,Modal_HSCN_2,Modal_Total_CN,Subclonal_HSCN_a1,Subclonal_HSCN_a2,Cancer_cell_frac_a1,Ccf_ci95_low_a1,Ccf_ci95_high_a1,Cancer_cell_frac_a2,Ccf_ci95_low_a2,Ccf_ci95_high_a2,LOH,Homozygous_deletion,new_solution
0,TCGA-OR-A5J1-01,1.0,1.0,1.0,2.0,0.0,0.0,0.05,0.02170,0.07127,0.05,0.02170,0.07127,0.0,0.0,1
1,TCGA-OR-A5J1-01,1.0,1.0,2.0,3.0,0.0,0.0,0.08,0.00356,0.14847,0.95,0.86228,1.00000,0.0,0.0,1
2,TCGA-OR-A5J1-01,1.0,1.0,1.0,2.0,0.0,0.0,0.06,0.02493,0.08426,0.06,0.02493,0.08426,0.0,0.0,1
3,TCGA-OR-A5J1-01,1.0,1.0,1.0,2.0,0.0,0.0,0.03,0.00000,0.04964,0.03,0.00000,0.04964,0.0,0.0,1
4,TCGA-OR-A5J1-01,1.0,1.0,1.0,2.0,0.0,0.0,0.04,0.01486,0.06423,0.04,0.01476,0.06412,0.0,0.0,1


### Normalizing data

In [17]:
abs_anttd_seg_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Chromosome,837726.0,9.263117,6.019021,1.00000,4.00000,8.00000,14.00000,22.00000
Modal_HSCN_1,837725.0,0.939152,0.692575,0.00000,1.00000,1.00000,1.00000,7.00000
Modal_HSCN_2,837725.0,1.873334,1.306222,0.00000,1.00000,1.00000,2.00000,7.00000
Modal_Total_CN,837725.0,2.761035,1.480728,0.00000,2.00000,2.00000,3.00000,7.00000
Subclonal_HSCN_a1,837725.0,0.148876,0.355966,0.00000,0.00000,0.00000,0.00000,1.00000
Subclonal_HSCN_a2,837725.0,0.198535,0.398898,0.00000,0.00000,0.00000,0.00000,1.00000
Cancer_cell_frac_a1,825889.0,0.348934,0.414076,0.00000,0.02000,0.08000,0.91000,1.00000
Ccf_ci95_low_a1,825889.0,0.263104,0.381182,0.00000,0.00000,0.00870,0.65862,0.99002
Ccf_ci95_high_a1,825889.0,0.430121,0.407173,0.01060,0.06846,0.20220,0.98158,1.00000
Cancer_cell_frac_a2,818002.0,0.217442,0.310762,0.00000,0.02000,0.06000,0.27000,1.00000


The data is not (well) normalized yet. All columns should have 0 mean and 1 standard deviation.

Save the dataframe before normalizing:

In [18]:
abs_anttd_seg_df.to_csv(f'{data_path}cleaned/unnormalized/copy_number_ratio.csv')

Normalize the data into a new dataframe:

In [19]:
# Exclude the categorical column(s)
categorical_cols = ['Chromosome']
numeric_cols = abs_anttd_seg_df.select_dtypes(include='number').columns.difference(categorical_cols)

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform numeric columns
abs_anttd_seg_df[numeric_cols] = scaler.fit_transform(abs_anttd_seg_df[numeric_cols])

# The categorical column stays unchanged
abs_anttd_seg_df.head()


,Sample,Chromosome,Modal_HSCN_1,Modal_HSCN_2,Modal_Total_CN,Subclonal_HSCN_a1,Subclonal_HSCN_a2,Cancer_cell_frac_a1,Ccf_ci95_low_a1,Ccf_ci95_high_a1,Cancer_cell_frac_a2,Ccf_ci95_low_a2,Ccf_ci95_high_a2,LOH,Homozygous_deletion,new_solution
0,TCGA-OR-A5J1-01,1.0,0.087858,-0.668596,-0.513960,-0.41823,-0.49771,-0.721931,-0.633305,-0.881324,-0.538811,-0.443579,-0.707777,-0.574648,-0.097583,0.001093
1,TCGA-OR-A5J1-01,1.0,0.087858,0.096971,0.161384,-0.41823,-0.49771,-0.649480,-0.680894,-0.691724,2.357300,2.636952,2.023483,-0.574648,-0.097583,0.001093
2,TCGA-OR-A5J1-01,1.0,0.087858,-0.668596,-0.513960,-0.41823,-0.49771,-0.697780,-0.624831,-0.849421,-0.506632,-0.431742,-0.669576,-0.574648,-0.097583,0.001093
3,TCGA-OR-A5J1-01,1.0,0.087858,-0.668596,-0.513960,-0.41823,-0.49771,-0.770231,-0.690233,-0.934447,-0.603169,-0.523104,-0.771388,-0.574648,-0.097583,0.001093
4,TCGA-OR-A5J1-01,1.0,0.087858,-0.668596,-0.513960,-0.41823,-0.49771,-0.746081,-0.651249,-0.898614,-0.570990,-0.469012,-0.728805,-0.574648,-0.097583,0.001093


Confirm that everything is ok through the `describe` method:

In [20]:
abs_anttd_seg_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Chromosome,837726.0,9.263117e+00,6.019021,1.000000,4.000000,8.000000,14.000000,22.000000
Modal_HSCN_1,837725.0,-1.998315e-17,1.000001,-1.356031,0.087858,0.087858,0.087858,8.751190
Modal_HSCN_2,837725.0,-5.835488e-18,1.000001,-1.434163,-0.668596,-0.668596,0.096971,3.924806
Modal_Total_CN,837725.0,-9.899973e-17,1.000001,-1.864649,-0.513960,-0.513960,0.161384,2.862760
Subclonal_HSCN_a1,837725.0,8.012769e-17,1.000001,-0.418230,-0.418230,-0.418230,-0.418230,2.391026
Subclonal_HSCN_a2,837725.0,-1.553868e-17,1.000001,-0.497710,-0.497710,-0.497710,-0.497710,2.009201
Cancer_cell_frac_a1,825889.0,8.011457e-17,1.000001,-0.842681,-0.794381,-0.649480,1.354985,1.572337
Ccf_ci95_low_a1,825889.0,3.411064e-16,1.000001,-0.690233,-0.690233,-0.667409,1.037604,1.907005
Ccf_ci95_high_a1,825889.0,4.523307e-16,1.000001,-1.030327,-0.888226,-0.559765,1.354361,1.399599
Cancer_cell_frac_a2,818002.0,-1.937744e-16,1.000001,-0.699706,-0.635348,-0.506632,0.169127,2.518195


### Aggregating sample data

#### Missing value imputation

We can't join rows correctly if there are missing values

In [21]:
nan_idx = abs_anttd_seg_df[abs_anttd_seg_df.Ccf_ci95_high_a2.isnull()].index
nan_idx

Index([   105,    106,    107,    114,    121,    123,    131,    132,    133,
          135,
       ...
       833845, 833846, 833847, 833848, 833851, 834453, 835054, 835276, 836216,
       837725],
      dtype='int64', length=19724)

In [22]:
abs_anttd_seg_df.iloc[nan_idx].head()

,Sample,Chromosome,Modal_HSCN_1,Modal_HSCN_2,Modal_Total_CN,Subclonal_HSCN_a1,Subclonal_HSCN_a2,Cancer_cell_frac_a1,Ccf_ci95_low_a1,Ccf_ci95_high_a1,Cancer_cell_frac_a2,Ccf_ci95_low_a2,Ccf_ci95_high_a2,LOH,Homozygous_deletion,new_solution
105,TCGA-OR-A5J1-01,5.0,0.087858,3.924806,2.86276,-0.41823,-0.49771,-0.770231,-0.689787,0.585792,NaN,NaN,NaN,-0.574648,-0.097583,0.001093
106,TCGA-OR-A5J1-01,5.0,0.087858,3.924806,2.86276,-0.41823,-0.49771,-0.649480,-0.669639,1.236155,NaN,NaN,NaN,-0.574648,-0.097583,0.001093
107,TCGA-OR-A5J1-01,5.0,0.087858,3.924806,2.86276,-0.41823,-0.49771,-0.697780,-0.690233,-0.166468,NaN,NaN,NaN,-0.574648,-0.097583,0.001093
114,TCGA-OR-A5J1-01,5.0,0.087858,3.924806,2.86276,-0.41823,-0.49771,-0.842681,-0.677851,1.148551,NaN,NaN,NaN,-0.574648,-0.097583,0.001093
121,TCGA-OR-A5J1-01,5.0,0.087858,3.924806,2.86276,-0.41823,-0.49771,-0.818531,-0.672630,1.228493,NaN,NaN,NaN,-0.574648,-0.097583,0.001093


In [23]:
abs_anttd_seg_df.head(125).tail(25)

,Sample,Chromosome,Modal_HSCN_1,Modal_HSCN_2,Modal_Total_CN,Subclonal_HSCN_a1,Subclonal_HSCN_a2,Cancer_cell_frac_a1,Ccf_ci95_low_a1,Ccf_ci95_high_a1,Cancer_cell_frac_a2,Ccf_ci95_low_a2,Ccf_ci95_high_a2,LOH,Homozygous_deletion,new_solution
100,TCGA-OR-A5J1-01,5.0,0.087858,0.096971,0.161384,-0.41823,-0.49771,-0.746081,-0.690233,-0.751502,2.486016,2.745576,2.023483,-0.574648,-0.097583,0.001093
101,TCGA-OR-A5J1-01,5.0,0.087858,0.862538,0.836728,-0.41823,-0.49771,-0.625330,-0.690233,-0.418916,1.906794,1.807102,1.901055,-0.574648,-0.097583,0.001093
102,TCGA-OR-A5J1-01,5.0,0.087858,0.862538,0.836728,-0.41823,-0.49771,-0.721931,-0.690233,-0.586241,2.325121,2.388811,2.023483,-0.574648,-0.097583,0.001093
103,TCGA-OR-A5J1-01,5.0,-1.356031,-0.668596,-1.189304,-0.41823,-0.49771,1.572337,1.834022,1.399599,-0.506632,-0.474290,-0.626051,1.740195,-0.097583,0.001093
104,TCGA-OR-A5J1-01,5.0,-1.356031,-0.668596,-1.189304,-0.41823,-0.49771,1.572337,1.878200,1.399599,-0.667527,-0.523104,-0.802385,1.740195,-0.097583,0.001093
105,TCGA-OR-A5J1-01,5.0,0.087858,3.924806,2.862760,-0.41823,-0.49771,-0.770231,-0.689787,0.585792,NaN,NaN,NaN,-0.574648,-0.097583,0.001093
106,TCGA-OR-A5J1-01,5.0,0.087858,3.924806,2.862760,-0.41823,-0.49771,-0.649480,-0.669639,1.236155,NaN,NaN,NaN,-0.574648,-0.097583,0.001093
107,TCGA-OR-A5J1-01,5.0,0.087858,3.924806,2.862760,-0.41823,-0.49771,-0.697780,-0.690233,-0.166468,NaN,NaN,NaN,-0.574648,-0.097583,0.001093
108,TCGA-OR-A5J1-01,5.0,0.087858,0.862538,0.836728,-0.41823,-0.49771,-0.818531,-0.690233,-0.734630,2.292942,2.453677,2.023483,-0.574648,-0.097583,0.001093
109,TCGA-OR-A5J1-01,5.0,-1.356031,-0.668596,-1.189304,-0.41823,-0.49771,1.572337,1.806712,1.399599,-0.538811,-0.521528,-0.665576,1.740195,-0.097583,0.001093


In [24]:
# Set index if 'Sample' should be preserved during interpolation
abs_anttd_seg_df = abs_anttd_seg_df.set_index('Sample')

# Interpolate numeric values (column-wise)
abs_anttd_seg_df = abs_anttd_seg_df.interpolate(method='linear', axis=0)

# Fill any remaining NaNs with forward fill, then backward fill
abs_anttd_seg_df = abs_anttd_seg_df.fillna(method='ffill', axis=0).fillna(method='bfill', axis=0)

# Reset index if needed
abs_anttd_seg_df = abs_anttd_seg_df.reset_index()

# Display the head
abs_anttd_seg_df.head()


,Sample,Chromosome,Modal_HSCN_1,Modal_HSCN_2,Modal_Total_CN,Subclonal_HSCN_a1,Subclonal_HSCN_a2,Cancer_cell_frac_a1,Ccf_ci95_low_a1,Ccf_ci95_high_a1,Cancer_cell_frac_a2,Ccf_ci95_low_a2,Ccf_ci95_high_a2,LOH,Homozygous_deletion,new_solution
0,TCGA-OR-A5J1-01,1.0,0.087858,-0.668596,-0.513960,-0.41823,-0.49771,-0.721931,-0.633305,-0.881324,-0.538811,-0.443579,-0.707777,-0.574648,-0.097583,0.001093
1,TCGA-OR-A5J1-01,1.0,0.087858,0.096971,0.161384,-0.41823,-0.49771,-0.649480,-0.680894,-0.691724,2.357300,2.636952,2.023483,-0.574648,-0.097583,0.001093
2,TCGA-OR-A5J1-01,1.0,0.087858,-0.668596,-0.513960,-0.41823,-0.49771,-0.697780,-0.624831,-0.849421,-0.506632,-0.431742,-0.669576,-0.574648,-0.097583,0.001093
3,TCGA-OR-A5J1-01,1.0,0.087858,-0.668596,-0.513960,-0.41823,-0.49771,-0.770231,-0.690233,-0.934447,-0.603169,-0.523104,-0.771388,-0.574648,-0.097583,0.001093
4,TCGA-OR-A5J1-01,1.0,0.087858,-0.668596,-0.513960,-0.41823,-0.49771,-0.746081,-0.651249,-0.898614,-0.570990,-0.469012,-0.728805,-0.574648,-0.097583,0.001093


In [25]:
# Percentage of missing values per column
missing_percent = abs_anttd_seg_df.isnull().mean().sort_values(ascending=False) * 100

# Display as a DataFrame
missing_percent_df = missing_percent[missing_percent > 0].to_frame(name='missing_percentage')
missing_percent_df


,missing_percentage


In [26]:
abs_anttd_seg_df.head(125).tail(25)

,Sample,Chromosome,Modal_HSCN_1,Modal_HSCN_2,Modal_Total_CN,Subclonal_HSCN_a1,Subclonal_HSCN_a2,Cancer_cell_frac_a1,Ccf_ci95_low_a1,Ccf_ci95_high_a1,Cancer_cell_frac_a2,Ccf_ci95_low_a2,Ccf_ci95_high_a2,LOH,Homozygous_deletion,new_solution
100,TCGA-OR-A5J1-01,5.0,0.087858,0.096971,0.161384,-0.41823,-0.49771,-0.746081,-0.690233,-0.751502,2.486016,2.745576,2.023483,-0.574648,-0.097583,0.001093
101,TCGA-OR-A5J1-01,5.0,0.087858,0.862538,0.836728,-0.41823,-0.49771,-0.625330,-0.690233,-0.418916,1.906794,1.807102,1.901055,-0.574648,-0.097583,0.001093
102,TCGA-OR-A5J1-01,5.0,0.087858,0.862538,0.836728,-0.41823,-0.49771,-0.721931,-0.690233,-0.586241,2.325121,2.388811,2.023483,-0.574648,-0.097583,0.001093
103,TCGA-OR-A5J1-01,5.0,-1.356031,-0.668596,-1.189304,-0.41823,-0.49771,1.572337,1.834022,1.399599,-0.506632,-0.474290,-0.626051,1.740195,-0.097583,0.001093
104,TCGA-OR-A5J1-01,5.0,-1.356031,-0.668596,-1.189304,-0.41823,-0.49771,1.572337,1.878200,1.399599,-0.667527,-0.523104,-0.802385,1.740195,-0.097583,0.001093
105,TCGA-OR-A5J1-01,5.0,0.087858,3.924806,2.862760,-0.41823,-0.49771,-0.770231,-0.689787,0.585792,0.072590,0.221091,-0.095918,-0.574648,-0.097583,0.001093
106,TCGA-OR-A5J1-01,5.0,0.087858,3.924806,2.862760,-0.41823,-0.49771,-0.649480,-0.669639,1.236155,0.812707,0.965286,0.610549,-0.574648,-0.097583,0.001093
107,TCGA-OR-A5J1-01,5.0,0.087858,3.924806,2.862760,-0.41823,-0.49771,-0.697780,-0.690233,-0.166468,1.552824,1.709482,1.317016,-0.574648,-0.097583,0.001093
108,TCGA-OR-A5J1-01,5.0,0.087858,0.862538,0.836728,-0.41823,-0.49771,-0.818531,-0.690233,-0.734630,2.292942,2.453677,2.023483,-0.574648,-0.097583,0.001093
109,TCGA-OR-A5J1-01,5.0,-1.356031,-0.668596,-1.189304,-0.41823,-0.49771,1.572337,1.806712,1.399599,-0.538811,-0.521528,-0.665576,1.740195,-0.097583,0.001093


#### Average groupby aggregation

Join all the data of each sample's chromosome through an average groupby aggregation:

In [27]:
abs_anttd_seg_df = abs_anttd_seg_df.groupby(['Sample', 'Chromosome']).mean()
abs_anttd_seg_df.head(25)

Modal_HSCN_1  Modal_HSCN_2  Modal_Total_CN  \
Sample          Chromosome                                               
TCGA-02-0003-01 1.0            -0.393438     -0.158218       -0.288846   
                2.0             0.087858     -0.413407       -0.288846   
                3.0             0.087858     -0.413407       -0.288846   
                4.0             0.087858     -0.668596       -0.513960   
                5.0             0.087858     -0.668596       -0.513960   
                6.0            -0.634086     -0.668596       -0.851632   
                7.0             0.087858     -0.209256       -0.108754   
                8.0             0.087858     -0.668596       -0.513960   
                9.0             0.087858     -0.668596       -0.513960   
                10.0           -1.356031     -0.668596       -1.189304   
                11.0            0.087858     -0.413407       -0.288846   
                12.0           -0.152790     -0.541001       -0.513960   
                13.0            0.087858     -0.668596       -0.513960   
                14.0            0.087858     -0.668596       -0.513960   
                15.0           -1.356031     -0.668596       -1.189304   
                16.0            0.087858     -0.668596       -0.513960   
                17.0            0.448830     -0.094421        0.161384   
                18.0            0.087858     -0.668596       -0.513960   
                19.0            0.087858     -0.668596       -0.513960   
                20.0            0.087858     -0.413407       -0.288846   
                21.0            0.087858     -0.668596       -0.513960   
                22.0            0.087858     -0.668596       -0.513960   
TCGA-02-0033-01 1.0             0.087858     -0.158218       -0.063731   
                2.0             0.087858     -0.668596       -0.513960   
                3.0             0.087858     -0.668596       -0.513960   

                            Subclonal_HSCN_a1  Subclonal_HSCN_a2  \
Sample          Chromosome                                         
TCGA-02-0003-01 1.0                 -0.418230          -0.497710   
                2.0                 -0.418230          -0.497710   
                3.0                 -0.418230          -0.497710   
                4.0                 -0.418230          -0.497710   
                5.0                 -0.418230          -0.497710   
                6.0                  0.986398          -0.497710   
                7.0                 -0.418230          -0.497710   
                8.0                 -0.418230          -0.497710   
                9.0                 -0.418230          -0.497710   
                10.0                -0.418230          -0.497710   
                11.0                -0.418230          -0.497710   
                12.0                -0.418230          -0.497710   
                13.0                -0.418230          -0.497710   
                14.0                -0.418230          -0.497710   
                15.0                -0.418230          -0.497710   
                16.0                -0.418230          -0.497710   
                17.0                -0.418230           0.755745   
                18.0                -0.418230          -0.497710   
                19.0                -0.418230          -0.497710   
                20.0                -0.418230          -0.497710   
                21.0                -0.418230          -0.497710   
                22.0                -0.418230          -0.497710   
TCGA-02-0033-01 1.0                 -0.418230          -0.497710   
                2.0                  2.391026           2.009201   
                3.0                 -0.418230          -0.497710   

                            Cancer_cell_frac_a1  Ccf_ci95_low_a1  \
Sample          Chromosome                                         
TCGA-02-0003-01 1.0                   -0.102076        -

#### Dividing chromosome data into different columns

Separate each chromosome's information into their own features.

OR

Create lists for each feature, containing each chromosome's value, and then apply an embedding bag on it.

In [28]:
abs_anttd_seg_df[abs_anttd_seg_df.index.get_level_values('Chromosome') == 1].head()

,,Modal_HSCN_1,Modal_HSCN_2,Modal_Total_CN,Subclonal_HSCN_a1,Subclonal_HSCN_a2,Cancer_cell_frac_a1,Ccf_ci95_low_a1,Ccf_ci95_high_a1,Cancer_cell_frac_a2,Ccf_ci95_low_a2,Ccf_ci95_high_a2,LOH,Homozygous_deletion,new_solution
Sample,Chromosome,,,,,,,,,,,,,,
TCGA-02-0003-01,1.0,-0.393438,-0.158218,-0.288846,-0.418230,-0.497710,-0.102076,-0.196864,-0.200884,-0.592443,-0.523104,-0.437729,0.196966,-0.097583,0.001093
TCGA-02-0033-01,1.0,0.087858,-0.158218,-0.063731,-0.418230,-0.497710,-0.794381,-0.690233,-0.636432,-0.624622,-0.523104,-0.401870,-0.574648,-0.097583,0.001093
TCGA-02-0047-01,1.0,-0.273114,-0.668596,-0.682796,1.688712,0.755745,-0.251002,-0.160276,-0.369643,-0.522722,-0.466584,-0.645049,0.004063,-0.097583,0.001093
TCGA-02-0055-01,1.0,0.087858,0.096971,0.161384,2.391026,0.337927,0.654630,0.801356,0.571294,-0.420822,-0.366900,-0.530503,-0.574648,-0.097583,0.001093
TCGA-02-2466-01,1.0,-0.118412,-0.559229,-0.513960,0.384414,-0.497710,-0.428678,-0.347539,-0.483315,-0.534214,-0.499451,-0.584329,-0.243956,-0.097583,0.001093


In [29]:
# List that will contain multiple dataframes, one for each chromosome
df_list = []
# Go through each chromosome and create its own dataframe, with properly labeled columns
for chrom in range(1, 23):
    # Filter for the current chromosome's dataframe
    tmp_df = abs_anttd_seg_df[abs_anttd_seg_df.index.get_level_values('Chromosome') == chrom]
    # Change the column names to identify the chromosome
    tmp_df.columns = [f'{col}_chromosome_{chrom}' for col in tmp_df.columns]
    # Remove now redundant `Chromosome` column
    tmp_df = tmp_df.reset_index().drop(columns='Chromosome', axis=1)
    # Add to the dataframes list
    df_list.append(tmp_df)

In [30]:
df_list[3]

,Sample,Modal_HSCN_1_chromosome_4,Modal_HSCN_2_chromosome_4,Modal_Total_CN_chromosome_4,Subclonal_HSCN_a1_chromosome_4,Subclonal_HSCN_a2_chromosome_4,Cancer_cell_frac_a1_chromosome_4,Ccf_ci95_low_a1_chromosome_4,Ccf_ci95_high_a1_chromosome_4,Cancer_cell_frac_a2_chromosome_4,Ccf_ci95_low_a2_chromosome_4,Ccf_ci95_high_a2_chromosome_4,LOH_chromosome_4,Homozygous_deletion_chromosome_4,new_solution_chromosome_4
0,TCGA-02-0003-01,0.087858,-0.668596,-0.513960,-0.418230,-0.497710,-0.818531,-0.690233,-0.993414,-0.667527,-0.523104,-0.836793,-0.574648,-0.097583,0.001093
1,TCGA-02-0033-01,0.087858,-0.668596,-0.513960,-0.418230,-0.497710,-0.818531,-0.690233,-0.940267,-0.667527,-0.523104,-0.778358,-0.574648,-0.097583,0.001093
2,TCGA-02-0047-01,-0.634086,-0.668596,-0.851632,0.986398,-0.497710,0.038800,0.116639,-0.028947,-0.538811,-0.519128,-0.610126,0.582774,-0.097583,0.001093
3,TCGA-02-0055-01,1.531747,0.096971,0.836728,2.391026,2.009201,-0.263077,-0.205582,-0.374561,0.072590,0.153925,-0.100961,-0.574648,-0.097583,0.001093
4,TCGA-02-2466-01,-0.596089,-0.708889,-0.869405,0.468903,-0.365768,0.272040,0.405542,0.142185,-0.425338,-0.310386,-0.544896,0.521857,0.446903,0.001093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4939,TCGA-ZJ-AB0H-01,0.643200,0.155861,0.473081,0.230060,0.080808,0.478148,0.527415,0.507897,-0.291280,-0.342598,-0.254802,-0.396583,-0.097583,0.001093
4940,TCGA-ZJ-AB0I-01,-0.791031,-0.701881,-0.954402,-0.296089,-0.170722,0.635730,0.724412,0.487527,-0.443673,-0.355755,-0.471342,0.834387,0.352209,0.001093
4941,TCGA-ZR-A9CJ-01,-0.374186,-0.086765,-0.297850,0.031251,0.304501,1.127008,0.990223,1.101770,0.269526,0.156388,0.382562,0.166102,-0.097583,0.001093
4942,TCGA-ZU-A8S4-01,-0.393438,-0.381508,-0.598378,-0.067073,-0.497710,-0.201695,-0.182774,-0.152428,-0.384620,-0.509769,-0.136065,0.196966,-0.097583,0.001093


In [31]:
abs_anttd_seg_df = reduce(lambda x, y: pandas.merge(x, y, on='Sample'), df_list)
abs_anttd_seg_df.head()

,Sample,Modal_HSCN_1_chromosome_1,Modal_HSCN_2_chromosome_1,Modal_Total_CN_chromosome_1,Subclonal_HSCN_a1_chromosome_1,Subclonal_HSCN_a2_chromosome_1,Cancer_cell_frac_a1_chromosome_1,Ccf_ci95_low_a1_chromosome_1,Ccf_ci95_high_a1_chromosome_1,Cancer_cell_frac_a2_chromosome_1,Ccf_ci95_low_a2_chromosome_1,Ccf_ci95_high_a2_chromosome_1,LOH_chromosome_1,Homozygous_deletion_chromosome_1,new_solution_chromosome_1,Modal_HSCN_1_chromosome_2,Modal_HSCN_2_chromosome_2,Modal_Total_CN_chromosome_2,Subclonal_HSCN_a1_chromosome_2,Subclonal_HSCN_a2_chromosome_2,Cancer_cell_frac_a1_chromosome_2,Ccf_ci95_low_a1_chromosome_2,Ccf_ci95_high_a1_chromosome_2,Cancer_cell_frac_a2_chromosome_2,Ccf_ci95_low_a2_chromosome_2,Ccf_ci95_high_a2_chromosome_2,LOH_chromosome_2,Homozygous_deletion_chromosome_2,new_solution_chromosome_2,Modal_HSCN_1_chromosome_3,Modal_HSCN_2_chromosome_3,Modal_Total_CN_chromosome_3,Subclonal_HSCN_a1_chromosome_3,Subclonal_HSCN_a2_chromosome_3,Cancer_cell_frac_a1_chromosome_3,Ccf_ci95_low_a1_chromosome_3,Ccf_ci95_high_a1_chromosome_3,Cancer_cell_frac_a2_chromosome_3,Ccf_ci95_low_a2_chromosome_3,Ccf_ci95_high_a2_chromosome_3,LOH_chromosome_3,Homozygous_deletion_chromosome_3,new_solution_chromosome_3,Modal_HSCN_1_chromosome_4,Modal_HSCN_2_chromosome_4,Modal_Total_CN_chromosome_4,Subclonal_HSCN_a1_chromosome_4,Subclonal_HSCN_a2_chromosome_4,Cancer_cell_frac_a1_chromosome_4,Ccf_ci95_low_a1_chromosome_4,Ccf_ci95_high_a1_chromosome_4,Cancer_cell_frac_a2_chromosome_4,Ccf_ci95_low_a2_chromosome_4,Ccf_ci95_high_a2_chromosome_4,LOH_chromosome_4,Homozygous_deletion_chromosome_4,new_solution_chromosome_4,Modal_HSCN_1_chromosome_5,Modal_HSCN_2_chromosome_5,Modal_Total_CN_chromosome_5,Subclonal_HSCN_a1_chromosome_5,Subclonal_HSCN_a2_chromosome_5,Cancer_cell_frac_a1_chromosome_5,Ccf_ci95_low_a1_chromosome_5,Ccf_ci95_high_a1_chromosome_5,Cancer_cell_frac_a2_chromosome_5,Ccf_ci95_low_a2_chromosome_5,Ccf_ci95_high_a2_chromosome_5,LOH_chromosome_5,Homozygous_deletion_chromosome_5,new_solution_chromosome_5,Modal_HSCN_1_chromosome_6,Modal_HSCN_2_chromosome_6,Modal_Total_CN_chromosome_6,Subclonal_HSCN_a1_chromosome_6,Subclonal_HSCN_a2_chromosome_6,Cancer_cell_frac_a1_chromosome_6,Ccf_ci95_low_a1_chromosome_6,Ccf_ci95_high_a1_chromosome_6,Cancer_cell_frac_a2_chromosome_6,Ccf_ci95_low_a2_chromosome_6,Ccf_ci95_high_a2_chromosome_6,LOH_chromosome_6,Homozygous_deletion_chromosome_6,new_solution_chromosome_6,Modal_HSCN_1_chromosome_7,Modal_HSCN_2_chromosome_7,Modal_Total_CN_chromosome_7,Subclonal_HSCN_a1_chromosome_7,Subclonal_HSCN_a2_chromosome_7,Cancer_cell_frac_a1_chromosome_7,Ccf_ci95_low_a1_chromosome_7,Ccf_ci95_high_a1_chromosome_7,Cancer_cell_frac_a2_chromosome_7,Ccf_ci95_low_a2_chromosome_7,Ccf_ci95_high_a2_chromosome_7,LOH_chromosome_7,Homozygous_deletion_chromosome_7,new_solution_chromosome_7,Modal_HSCN_1_chromosome_8,Modal_HSCN_2_chromosome_8,Modal_Total_CN_chromosome_8,Subclonal_HSCN_a1_chromosome_8,Subclonal_HSCN_a2_chromosome_8,Cancer_cell_frac_a1_chromosome_8,Ccf_ci95_low_a1_chromosome_8,Ccf_ci95_high_a1_chromosome_8,Cancer_cell_frac_a2_chromosome_8,Ccf_ci95_low_a2_chromosome_8,Ccf_ci95_high_a2_chromosome_8,LOH_chromosome_8,Homozygous_deletion_chromosome_8,new_solution_chromosome_8,Modal_HSCN_1_chromosome_9,Modal_HSCN_2_chromosome_9,Modal_Total_CN_chromosome_9,Subclonal_HSCN_a1_chromosome_9,Subclonal_HSCN_a2_chromosome_9,Cancer_cell_frac_a1_chromosome_9,Ccf_ci95_low_a1_chromosome_9,Ccf_ci95_high_a1_chromosome_9,Cancer_cell_frac_a2_chromosome_9,Ccf_ci95_low_a2_chromosome_9,Ccf_ci95_high_a2_chromosome_9,LOH_chromosome_9,Homozygous_deletion_chromosome_9,new_solution_chromosome_9,Modal_HSCN_1_chromosome_10,Modal_HSCN_2_chromosome_10,Modal_Total_CN_chromosome_10,Subclonal_HSCN_a1_chromosome_10,Subclonal_HSCN_a2_chromosome_10,Cancer_cell_frac_a1_chromosome_10,Ccf_ci95_low_a1_chromosome_10,Ccf_ci95_high_a1_chromosome_10,Cancer_cell_frac_a2_chromosome_10,Ccf_ci95_low_a2_chromosome_10,Ccf_ci95_high_a2_chromosome_10,LOH_chromosome_10,Homozygous_deletion

In [32]:
abs_anttd_seg_df.Sample.nunique()

4943

In [33]:
len(abs_anttd_seg_df)

4943

Remove duplicate columns (redundant features that are independent of the chromosome):

In [34]:
unique_features = set([col.split('_chromosome')[0] for col in abs_anttd_seg_df.columns])
unique_features

{'Cancer_cell_frac_a1',
 'Cancer_cell_frac_a2',
 'Ccf_ci95_high_a1',
 'Ccf_ci95_high_a2',
 'Ccf_ci95_low_a1',
 'Ccf_ci95_low_a2',
 'Homozygous_deletion',
 'LOH',
 'Modal_HSCN_1',
 'Modal_HSCN_2',
 'Modal_Total_CN',
 'Sample',
 'Subclonal_HSCN_a1',
 'Subclonal_HSCN_a2',
 'new_solution'}

In [35]:
[col for col in abs_anttd_seg_df.columns if 'Cancer_cell_frac_a1' in col]

['Cancer_cell_frac_a1_chromosome_1',
 'Cancer_cell_frac_a1_chromosome_2',
 'Cancer_cell_frac_a1_chromosome_3',
 'Cancer_cell_frac_a1_chromosome_4',
 'Cancer_cell_frac_a1_chromosome_5',
 'Cancer_cell_frac_a1_chromosome_6',
 'Cancer_cell_frac_a1_chromosome_7',
 'Cancer_cell_frac_a1_chromosome_8',
 'Cancer_cell_frac_a1_chromosome_9',
 'Cancer_cell_frac_a1_chromosome_10',
 'Cancer_cell_frac_a1_chromosome_11',
 'Cancer_cell_frac_a1_chromosome_12',
 'Cancer_cell_frac_a1_chromosome_13',
 'Cancer_cell_frac_a1_chromosome_14',
 'Cancer_cell_frac_a1_chromosome_15',
 'Cancer_cell_frac_a1_chromosome_16',
 'Cancer_cell_frac_a1_chromosome_17',
 'Cancer_cell_frac_a1_chromosome_18',
 'Cancer_cell_frac_a1_chromosome_19',
 'Cancer_cell_frac_a1_chromosome_20',
 'Cancer_cell_frac_a1_chromosome_21',
 'Cancer_cell_frac_a1_chromosome_22']

In [37]:
# Save the feature names that are redundant (i.e. no difference between chromosomes)
redundant_features = []

for feature in unique_features:
    # Flag that indicates if all of the feature's columns are equal
    all_cols_equal = True
    # List of column names that are part of the same unique feature
    chrom_cols = [col for col in abs_anttd_seg_df.columns if feature in col]
    
    for i in range(len(chrom_cols) - 1):
        # Check if the current pair of columns are completely equal
        if not (abs_anttd_seg_df[chrom_cols[i]] == abs_anttd_seg_df[chrom_cols[i+1]]).all():
            all_cols_equal = False
            break

    if all_cols_equal:
        redundant_features.append(feature)

redundant_features


['new_solution', 'Sample']

In [38]:
column_duplicates = [col for col in abs_anttd_seg_df.columns if 'new_solution' in col]
column_duplicates.remove('new_solution_chromosome_1')
column_duplicates

['new_solution_chromosome_2',
 'new_solution_chromosome_3',
 'new_solution_chromosome_4',
 'new_solution_chromosome_5',
 'new_solution_chromosome_6',
 'new_solution_chromosome_7',
 'new_solution_chromosome_8',
 'new_solution_chromosome_9',
 'new_solution_chromosome_10',
 'new_solution_chromosome_11',
 'new_solution_chromosome_12',
 'new_solution_chromosome_13',
 'new_solution_chromosome_14',
 'new_solution_chromosome_15',
 'new_solution_chromosome_16',
 'new_solution_chromosome_17',
 'new_solution_chromosome_18',
 'new_solution_chromosome_19',
 'new_solution_chromosome_20',
 'new_solution_chromosome_21',
 'new_solution_chromosome_22']

In [39]:
abs_anttd_seg_df = abs_anttd_seg_df.drop(columns=column_duplicates, axis=1)
abs_anttd_seg_df = abs_anttd_seg_df.rename(columns={'new_solution_chromosome_1': 'new_solution'})
abs_anttd_seg_df.head()

,Sample,Modal_HSCN_1_chromosome_1,Modal_HSCN_2_chromosome_1,Modal_Total_CN_chromosome_1,Subclonal_HSCN_a1_chromosome_1,Subclonal_HSCN_a2_chromosome_1,Cancer_cell_frac_a1_chromosome_1,Ccf_ci95_low_a1_chromosome_1,Ccf_ci95_high_a1_chromosome_1,Cancer_cell_frac_a2_chromosome_1,Ccf_ci95_low_a2_chromosome_1,Ccf_ci95_high_a2_chromosome_1,LOH_chromosome_1,Homozygous_deletion_chromosome_1,new_solution,Modal_HSCN_1_chromosome_2,Modal_HSCN_2_chromosome_2,Modal_Total_CN_chromosome_2,Subclonal_HSCN_a1_chromosome_2,Subclonal_HSCN_a2_chromosome_2,Cancer_cell_frac_a1_chromosome_2,Ccf_ci95_low_a1_chromosome_2,Ccf_ci95_high_a1_chromosome_2,Cancer_cell_frac_a2_chromosome_2,Ccf_ci95_low_a2_chromosome_2,Ccf_ci95_high_a2_chromosome_2,LOH_chromosome_2,Homozygous_deletion_chromosome_2,Modal_HSCN_1_chromosome_3,Modal_HSCN_2_chromosome_3,Modal_Total_CN_chromosome_3,Subclonal_HSCN_a1_chromosome_3,Subclonal_HSCN_a2_chromosome_3,Cancer_cell_frac_a1_chromosome_3,Ccf_ci95_low_a1_chromosome_3,Ccf_ci95_high_a1_chromosome_3,Cancer_cell_frac_a2_chromosome_3,Ccf_ci95_low_a2_chromosome_3,Ccf_ci95_high_a2_chromosome_3,LOH_chromosome_3,Homozygous_deletion_chromosome_3,Modal_HSCN_1_chromosome_4,Modal_HSCN_2_chromosome_4,Modal_Total_CN_chromosome_4,Subclonal_HSCN_a1_chromosome_4,Subclonal_HSCN_a2_chromosome_4,Cancer_cell_frac_a1_chromosome_4,Ccf_ci95_low_a1_chromosome_4,Ccf_ci95_high_a1_chromosome_4,Cancer_cell_frac_a2_chromosome_4,Ccf_ci95_low_a2_chromosome_4,Ccf_ci95_high_a2_chromosome_4,LOH_chromosome_4,Homozygous_deletion_chromosome_4,Modal_HSCN_1_chromosome_5,Modal_HSCN_2_chromosome_5,Modal_Total_CN_chromosome_5,Subclonal_HSCN_a1_chromosome_5,Subclonal_HSCN_a2_chromosome_5,Cancer_cell_frac_a1_chromosome_5,Ccf_ci95_low_a1_chromosome_5,Ccf_ci95_high_a1_chromosome_5,Cancer_cell_frac_a2_chromosome_5,Ccf_ci95_low_a2_chromosome_5,Ccf_ci95_high_a2_chromosome_5,LOH_chromosome_5,Homozygous_deletion_chromosome_5,Modal_HSCN_1_chromosome_6,Modal_HSCN_2_chromosome_6,Modal_Total_CN_chromosome_6,Subclonal_HSCN_a1_chromosome_6,Subclonal_HSCN_a2_chromosome_6,Cancer_cell_frac_a1_chromosome_6,Ccf_ci95_low_a1_chromosome_6,Ccf_ci95_high_a1_chromosome_6,Cancer_cell_frac_a2_chromosome_6,Ccf_ci95_low_a2_chromosome_6,Ccf_ci95_high_a2_chromosome_6,LOH_chromosome_6,Homozygous_deletion_chromosome_6,Modal_HSCN_1_chromosome_7,Modal_HSCN_2_chromosome_7,Modal_Total_CN_chromosome_7,Subclonal_HSCN_a1_chromosome_7,Subclonal_HSCN_a2_chromosome_7,Cancer_cell_frac_a1_chromosome_7,Ccf_ci95_low_a1_chromosome_7,Ccf_ci95_high_a1_chromosome_7,Cancer_cell_frac_a2_chromosome_7,Ccf_ci95_low_a2_chromosome_7,Ccf_ci95_high_a2_chromosome_7,LOH_chromosome_7,Homozygous_deletion_chromosome_7,Modal_HSCN_1_chromosome_8,Modal_HSCN_2_chromosome_8,Modal_Total_CN_chromosome_8,Subclonal_HSCN_a1_chromosome_8,Subclonal_HSCN_a2_chromosome_8,Cancer_cell_frac_a1_chromosome_8,Ccf_ci95_low_a1_chromosome_8,Ccf_ci95_high_a1_chromosome_8,Cancer_cell_frac_a2_chromosome_8,Ccf_ci95_low_a2_chromosome_8,Ccf_ci95_high_a2_chromosome_8,LOH_chromosome_8,Homozygous_deletion_chromosome_8,Modal_HSCN_1_chromosome_9,Modal_HSCN_2_chromosome_9,Modal_Total_CN_chromosome_9,Subclonal_HSCN_a1_chromosome_9,Subclonal_HSCN_a2_chromosome_9,Cancer_cell_frac_a1_chromosome_9,Ccf_ci95_low_a1_chromosome_9,Ccf_ci95_high_a1_chromosome_9,Cancer_cell_frac_a2_chromosome_9,Ccf_ci95_low_a2_chromosome_9,Ccf_ci95_high_a2_chromosome_9,LOH_chromosome_9,Homozygous_deletion_chromosome_9,Modal_HSCN_1_chromosome_10,Modal_HSCN_2_chromosome_10,Modal_Total_CN_chromosome_10,Subclonal_HSCN_a1_chromosome_10,Subclonal_HSCN_a2_chromosome_10,Cancer_cell_frac_a1_chromosome_10,Ccf_ci95_low_a1_chromosome_10,Ccf_ci95_high_a1_chromosome_10,Cancer_cell_frac_a2_chromosome_10,Ccf_ci95_low_a2_chromosome_10,Ccf_ci95_high_a2_chromosome_10,LOH_chromosome_10,Homozygous_deletion_chromosome_10,Modal_HSCN_1_chromosome_11,Modal_HSCN_2_chromosome_11,Modal_Total_CN_chromosome_11,Subclonal_HSCN_a1_chromosome_11,Subclonal_HSCN_a2_chromosome_11,Cancer_cell_frac_a1_chromosome_11,Ccf_ci95_low_a1_chromosom

In [40]:
[col for col in abs_anttd_seg_df.columns if 'new_solution' in col]

['new_solution']

### Setting the index

Set `sample_id` column to be the index:

In [41]:
abs_anttd_seg_df = abs_anttd_seg_df.set_index('Sample')
abs_anttd_seg_df.head()

,Modal_HSCN_1_chromosome_1,Modal_HSCN_2_chromosome_1,Modal_Total_CN_chromosome_1,Subclonal_HSCN_a1_chromosome_1,Subclonal_HSCN_a2_chromosome_1,Cancer_cell_frac_a1_chromosome_1,Ccf_ci95_low_a1_chromosome_1,Ccf_ci95_high_a1_chromosome_1,Cancer_cell_frac_a2_chromosome_1,Ccf_ci95_low_a2_chromosome_1,Ccf_ci95_high_a2_chromosome_1,LOH_chromosome_1,Homozygous_deletion_chromosome_1,new_solution,Modal_HSCN_1_chromosome_2,Modal_HSCN_2_chromosome_2,Modal_Total_CN_chromosome_2,Subclonal_HSCN_a1_chromosome_2,Subclonal_HSCN_a2_chromosome_2,Cancer_cell_frac_a1_chromosome_2,Ccf_ci95_low_a1_chromosome_2,Ccf_ci95_high_a1_chromosome_2,Cancer_cell_frac_a2_chromosome_2,Ccf_ci95_low_a2_chromosome_2,Ccf_ci95_high_a2_chromosome_2,LOH_chromosome_2,Homozygous_deletion_chromosome_2,Modal_HSCN_1_chromosome_3,Modal_HSCN_2_chromosome_3,Modal_Total_CN_chromosome_3,Subclonal_HSCN_a1_chromosome_3,Subclonal_HSCN_a2_chromosome_3,Cancer_cell_frac_a1_chromosome_3,Ccf_ci95_low_a1_chromosome_3,Ccf_ci95_high_a1_chromosome_3,Cancer_cell_frac_a2_chromosome_3,Ccf_ci95_low_a2_chromosome_3,Ccf_ci95_high_a2_chromosome_3,LOH_chromosome_3,Homozygous_deletion_chromosome_3,Modal_HSCN_1_chromosome_4,Modal_HSCN_2_chromosome_4,Modal_Total_CN_chromosome_4,Subclonal_HSCN_a1_chromosome_4,Subclonal_HSCN_a2_chromosome_4,Cancer_cell_frac_a1_chromosome_4,Ccf_ci95_low_a1_chromosome_4,Ccf_ci95_high_a1_chromosome_4,Cancer_cell_frac_a2_chromosome_4,Ccf_ci95_low_a2_chromosome_4,Ccf_ci95_high_a2_chromosome_4,LOH_chromosome_4,Homozygous_deletion_chromosome_4,Modal_HSCN_1_chromosome_5,Modal_HSCN_2_chromosome_5,Modal_Total_CN_chromosome_5,Subclonal_HSCN_a1_chromosome_5,Subclonal_HSCN_a2_chromosome_5,Cancer_cell_frac_a1_chromosome_5,Ccf_ci95_low_a1_chromosome_5,Ccf_ci95_high_a1_chromosome_5,Cancer_cell_frac_a2_chromosome_5,Ccf_ci95_low_a2_chromosome_5,Ccf_ci95_high_a2_chromosome_5,LOH_chromosome_5,Homozygous_deletion_chromosome_5,Modal_HSCN_1_chromosome_6,Modal_HSCN_2_chromosome_6,Modal_Total_CN_chromosome_6,Subclonal_HSCN_a1_chromosome_6,Subclonal_HSCN_a2_chromosome_6,Cancer_cell_frac_a1_chromosome_6,Ccf_ci95_low_a1_chromosome_6,Ccf_ci95_high_a1_chromosome_6,Cancer_cell_frac_a2_chromosome_6,Ccf_ci95_low_a2_chromosome_6,Ccf_ci95_high_a2_chromosome_6,LOH_chromosome_6,Homozygous_deletion_chromosome_6,Modal_HSCN_1_chromosome_7,Modal_HSCN_2_chromosome_7,Modal_Total_CN_chromosome_7,Subclonal_HSCN_a1_chromosome_7,Subclonal_HSCN_a2_chromosome_7,Cancer_cell_frac_a1_chromosome_7,Ccf_ci95_low_a1_chromosome_7,Ccf_ci95_high_a1_chromosome_7,Cancer_cell_frac_a2_chromosome_7,Ccf_ci95_low_a2_chromosome_7,Ccf_ci95_high_a2_chromosome_7,LOH_chromosome_7,Homozygous_deletion_chromosome_7,Modal_HSCN_1_chromosome_8,Modal_HSCN_2_chromosome_8,Modal_Total_CN_chromosome_8,Subclonal_HSCN_a1_chromosome_8,Subclonal_HSCN_a2_chromosome_8,Cancer_cell_frac_a1_chromosome_8,Ccf_ci95_low_a1_chromosome_8,Ccf_ci95_high_a1_chromosome_8,Cancer_cell_frac_a2_chromosome_8,Ccf_ci95_low_a2_chromosome_8,Ccf_ci95_high_a2_chromosome_8,LOH_chromosome_8,Homozygous_deletion_chromosome_8,Modal_HSCN_1_chromosome_9,Modal_HSCN_2_chromosome_9,Modal_Total_CN_chromosome_9,Subclonal_HSCN_a1_chromosome_9,Subclonal_HSCN_a2_chromosome_9,Cancer_cell_frac_a1_chromosome_9,Ccf_ci95_low_a1_chromosome_9,Ccf_ci95_high_a1_chromosome_9,Cancer_cell_frac_a2_chromosome_9,Ccf_ci95_low_a2_chromosome_9,Ccf_ci95_high_a2_chromosome_9,LOH_chromosome_9,Homozygous_deletion_chromosome_9,Modal_HSCN_1_chromosome_10,Modal_HSCN_2_chromosome_10,Modal_Total_CN_chromosome_10,Subclonal_HSCN_a1_chromosome_10,Subclonal_HSCN_a2_chromosome_10,Cancer_cell_frac_a1_chromosome_10,Ccf_ci95_low_a1_chromosome_10,Ccf_ci95_high_a1_chromosome_10,Cancer_cell_frac_a2_chromosome_10,Ccf_ci95_low_a2_chromosome_10,Ccf_ci95_high_a2_chromosome_10,LOH_chromosome_10,Homozygous_deletion_chromosome_10,Modal_HSCN_1_chromosome_11,Modal_HSCN_2_chromosome_11,Modal_Total_CN_chromosome_11,Subclonal_HSCN_a1_chromosome_11,Subclonal_HSCN_a2_chromosome_11,Cancer_cell_frac_a1_chromosome_11,Ccf_ci95_low_a1_chromosome_11,Cc

Fix the index name:

In [44]:
abs_anttd_seg_df.index.name = 'sample_id'
abs_anttd_seg_df.head()


,Modal_HSCN_1_chromosome_1,Modal_HSCN_2_chromosome_1,Modal_Total_CN_chromosome_1,Subclonal_HSCN_a1_chromosome_1,Subclonal_HSCN_a2_chromosome_1,Cancer_cell_frac_a1_chromosome_1,Ccf_ci95_low_a1_chromosome_1,Ccf_ci95_high_a1_chromosome_1,Cancer_cell_frac_a2_chromosome_1,Ccf_ci95_low_a2_chromosome_1,Ccf_ci95_high_a2_chromosome_1,LOH_chromosome_1,Homozygous_deletion_chromosome_1,new_solution,Modal_HSCN_1_chromosome_2,Modal_HSCN_2_chromosome_2,Modal_Total_CN_chromosome_2,Subclonal_HSCN_a1_chromosome_2,Subclonal_HSCN_a2_chromosome_2,Cancer_cell_frac_a1_chromosome_2,Ccf_ci95_low_a1_chromosome_2,Ccf_ci95_high_a1_chromosome_2,Cancer_cell_frac_a2_chromosome_2,Ccf_ci95_low_a2_chromosome_2,Ccf_ci95_high_a2_chromosome_2,LOH_chromosome_2,Homozygous_deletion_chromosome_2,Modal_HSCN_1_chromosome_3,Modal_HSCN_2_chromosome_3,Modal_Total_CN_chromosome_3,Subclonal_HSCN_a1_chromosome_3,Subclonal_HSCN_a2_chromosome_3,Cancer_cell_frac_a1_chromosome_3,Ccf_ci95_low_a1_chromosome_3,Ccf_ci95_high_a1_chromosome_3,Cancer_cell_frac_a2_chromosome_3,Ccf_ci95_low_a2_chromosome_3,Ccf_ci95_high_a2_chromosome_3,LOH_chromosome_3,Homozygous_deletion_chromosome_3,Modal_HSCN_1_chromosome_4,Modal_HSCN_2_chromosome_4,Modal_Total_CN_chromosome_4,Subclonal_HSCN_a1_chromosome_4,Subclonal_HSCN_a2_chromosome_4,Cancer_cell_frac_a1_chromosome_4,Ccf_ci95_low_a1_chromosome_4,Ccf_ci95_high_a1_chromosome_4,Cancer_cell_frac_a2_chromosome_4,Ccf_ci95_low_a2_chromosome_4,Ccf_ci95_high_a2_chromosome_4,LOH_chromosome_4,Homozygous_deletion_chromosome_4,Modal_HSCN_1_chromosome_5,Modal_HSCN_2_chromosome_5,Modal_Total_CN_chromosome_5,Subclonal_HSCN_a1_chromosome_5,Subclonal_HSCN_a2_chromosome_5,Cancer_cell_frac_a1_chromosome_5,Ccf_ci95_low_a1_chromosome_5,Ccf_ci95_high_a1_chromosome_5,Cancer_cell_frac_a2_chromosome_5,Ccf_ci95_low_a2_chromosome_5,Ccf_ci95_high_a2_chromosome_5,LOH_chromosome_5,Homozygous_deletion_chromosome_5,Modal_HSCN_1_chromosome_6,Modal_HSCN_2_chromosome_6,Modal_Total_CN_chromosome_6,Subclonal_HSCN_a1_chromosome_6,Subclonal_HSCN_a2_chromosome_6,Cancer_cell_frac_a1_chromosome_6,Ccf_ci95_low_a1_chromosome_6,Ccf_ci95_high_a1_chromosome_6,Cancer_cell_frac_a2_chromosome_6,Ccf_ci95_low_a2_chromosome_6,Ccf_ci95_high_a2_chromosome_6,LOH_chromosome_6,Homozygous_deletion_chromosome_6,Modal_HSCN_1_chromosome_7,Modal_HSCN_2_chromosome_7,Modal_Total_CN_chromosome_7,Subclonal_HSCN_a1_chromosome_7,Subclonal_HSCN_a2_chromosome_7,Cancer_cell_frac_a1_chromosome_7,Ccf_ci95_low_a1_chromosome_7,Ccf_ci95_high_a1_chromosome_7,Cancer_cell_frac_a2_chromosome_7,Ccf_ci95_low_a2_chromosome_7,Ccf_ci95_high_a2_chromosome_7,LOH_chromosome_7,Homozygous_deletion_chromosome_7,Modal_HSCN_1_chromosome_8,Modal_HSCN_2_chromosome_8,Modal_Total_CN_chromosome_8,Subclonal_HSCN_a1_chromosome_8,Subclonal_HSCN_a2_chromosome_8,Cancer_cell_frac_a1_chromosome_8,Ccf_ci95_low_a1_chromosome_8,Ccf_ci95_high_a1_chromosome_8,Cancer_cell_frac_a2_chromosome_8,Ccf_ci95_low_a2_chromosome_8,Ccf_ci95_high_a2_chromosome_8,LOH_chromosome_8,Homozygous_deletion_chromosome_8,Modal_HSCN_1_chromosome_9,Modal_HSCN_2_chromosome_9,Modal_Total_CN_chromosome_9,Subclonal_HSCN_a1_chromosome_9,Subclonal_HSCN_a2_chromosome_9,Cancer_cell_frac_a1_chromosome_9,Ccf_ci95_low_a1_chromosome_9,Ccf_ci95_high_a1_chromosome_9,Cancer_cell_frac_a2_chromosome_9,Ccf_ci95_low_a2_chromosome_9,Ccf_ci95_high_a2_chromosome_9,LOH_chromosome_9,Homozygous_deletion_chromosome_9,Modal_HSCN_1_chromosome_10,Modal_HSCN_2_chromosome_10,Modal_Total_CN_chromosome_10,Subclonal_HSCN_a1_chromosome_10,Subclonal_HSCN_a2_chromosome_10,Cancer_cell_frac_a1_chromosome_10,Ccf_ci95_low_a1_chromosome_10,Ccf_ci95_high_a1_chromosome_10,Cancer_cell_frac_a2_chromosome_10,Ccf_ci95_low_a2_chromosome_10,Ccf_ci95_high_a2_chromosome_10,LOH_chromosome_10,Homozygous_deletion_chromosome_10,Modal_HSCN_1_chromosome_11,Modal_HSCN_2_chromosome_11,Modal_Total_CN_chromosome_11,Subclonal_HSCN_a1_chromosome_11,Subclonal_HSCN_a2_chromosome_11,Cancer_cell_frac_a1_chromosome_11,Ccf_ci95_low_a1_chromosome_11,Cc

Save the normalized dataframe:

In [45]:
abs_anttd_seg_df.to_csv(f'{data_path}cleaned/normalized/copy_number_ratio.csv')

## Clinical outcome (TCGA-CDR) data

Description

### Loading the data

In [46]:
cdr_df.dtypes

Unnamed: 0                               int64
bcr_patient_barcode                     object
type                                    object
age_at_initial_pathologic_diagnosis    float64
gender                                  object
race                                    object
ajcc_pathologic_tumor_stage             object
clinical_stage                          object
histological_type                       object
histological_grade                      object
initial_pathologic_dx_year             float64
menopause_status                        object
birth_days_to                          float64
vital_status                            object
tumor_status                            object
last_contact_days_to                   float64
death_days_to                          float64
cause_of_death                          object
new_tumor_event_type                    object
new_tumor_event_site                    object
new_tumor_event_site_other              object
new_tumor_eve

### Setting the index

Set `sample_id` column to be the index:

In [47]:
cdr_df = cdr_df.set_index('bcr_patient_barcode')
cdr_df.head()

,Unnamed: 0,type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,initial_pathologic_dx_year,menopause_status,birth_days_to,vital_status,tumor_status,last_contact_days_to,death_days_to,cause_of_death,new_tumor_event_type,new_tumor_event_site,new_tumor_event_site_other,new_tumor_event_dx_days_to,treatment_outcome_first_course,margin_status,residual_tumor,OS,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction
bcr_patient_barcode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TCGA-OR-A5J1,1,ACC,58.0,MALE,WHITE,Stage II,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,[Not Available],-21496.0,Dead,WITH TUMOR,NaN,1355.0,[Not Available],Distant Metastasis,Peritoneal Surfaces,NaN,754.0,Complete Remission/Response,NaN,NaN,1.0,1355.0,1.0,1355.0,1.0,754.0,1.0,754.0,NaN
TCGA-OR-A5J2,2,ACC,44.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2004.0,[Not Available],-16090.0,Dead,WITH TUMOR,NaN,1677.0,[Not Available],Distant Metastasis,Soft Tissue,NaN,289.0,Progressive Disease,NaN,NaN,1.0,1677.0,1.0,1677.0,NaN,NaN,1.0,289.0,NaN
TCGA-OR-A5J3,3,ACC,23.0,FEMALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2008.0,[Not Available],-8624.0,Alive,WITH TUMOR,2091.0,NaN,[Not Available],Distant Metastasis,Lung,NaN,53.0,Complete Remission/Response,NaN,NaN,0.0,2091.0,0.0,2091.0,1.0,53.0,1.0,53.0,NaN
TCGA-OR-A5J4,4,ACC,23.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,[Not Available],-8451.0,Dead,WITH TUMOR,NaN,423.0,[Not Available],Locoregional Recurrence,Peritoneal Surfaces,NaN,126.0,Progressive Disease,NaN,NaN,1.0,423.0,1.0,423.0,NaN,NaN,1.0,126.0,NaN
TCGA-OR-A5J5,5,ACC,30.0,MALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,[Not Available],-11171.0,Dead,WITH TUMOR,NaN,365.0,[Not Available],Locoregional Recurrence,"Other, specify",vena cava thrombus,50.0,Progressive Disease,NaN,NaN,1.0,365.0,1.0,365.0,NaN,NaN,1.0,50.0,NaN


Fix the index name:

In [51]:
cdr_df.index.name = 'sample_id'
cdr_df.head()


,Unnamed: 0,type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,initial_pathologic_dx_year,menopause_status,birth_days_to,vital_status,tumor_status,last_contact_days_to,death_days_to,cause_of_death,new_tumor_event_type,new_tumor_event_site,new_tumor_event_site_other,new_tumor_event_dx_days_to,treatment_outcome_first_course,margin_status,residual_tumor,OS,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction
sample_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TCGA-OR-A5J1,1,ACC,58.0,MALE,WHITE,Stage II,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,[Not Available],-21496.0,Dead,WITH TUMOR,NaN,1355.0,[Not Available],Distant Metastasis,Peritoneal Surfaces,NaN,754.0,Complete Remission/Response,NaN,NaN,1.0,1355.0,1.0,1355.0,1.0,754.0,1.0,754.0,NaN
TCGA-OR-A5J2,2,ACC,44.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2004.0,[Not Available],-16090.0,Dead,WITH TUMOR,NaN,1677.0,[Not Available],Distant Metastasis,Soft Tissue,NaN,289.0,Progressive Disease,NaN,NaN,1.0,1677.0,1.0,1677.0,NaN,NaN,1.0,289.0,NaN
TCGA-OR-A5J3,3,ACC,23.0,FEMALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2008.0,[Not Available],-8624.0,Alive,WITH TUMOR,2091.0,NaN,[Not Available],Distant Metastasis,Lung,NaN,53.0,Complete Remission/Response,NaN,NaN,0.0,2091.0,0.0,2091.0,1.0,53.0,1.0,53.0,NaN
TCGA-OR-A5J4,4,ACC,23.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,[Not Available],-8451.0,Dead,WITH TUMOR,NaN,423.0,[Not Available],Locoregional Recurrence,Peritoneal Surfaces,NaN,126.0,Progressive Disease,NaN,NaN,1.0,423.0,1.0,423.0,NaN,NaN,1.0,126.0,NaN
TCGA-OR-A5J5,5,ACC,30.0,MALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,[Not Available],-11171.0,Dead,WITH TUMOR,NaN,365.0,[Not Available],Locoregional Recurrence,"Other, specify",vena cava thrombus,50.0,Progressive Disease,NaN,NaN,1.0,365.0,1.0,365.0,NaN,NaN,1.0,50.0,NaN


### Checking for missing values

Considerable percentage of missing values on `ajcc_pathologic_tumor_stage` (\~37%) and `clinical_stage` (\~76%).
Considering the real percentages of missing values, which are higher than what we got before standardizing the missing values representation, the main features to use from this table should be `gender`, `vital_status`, `age_at_initial_pathologic_diagnosis`, `tumor_status`, `race` and `ajcc_pathologic_tumor_stage`.

### Removing unneeded features

Remove columns that have more than 40% missing values:

In [52]:
# Set the threshold percentage
nan_percent_thrsh = 40  # percent
threshold = nan_percent_thrsh / 100.0

# Drop columns with more than threshold% NaNs
cdr_df = cdr_df.loc[:, cdr_df.isnull().mean() <= threshold]


In [54]:
# Show count and percentage of missing values per column
missing_values = cdr_df.isnull().sum()
missing_percent = (missing_values / len(cdr_df)) * 100

missing_df = pandas.DataFrame({
    'Missing Count': missing_values,
    'Missing Percent': missing_percent
})

# Display only columns with missing values
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values(by='Missing Percent', ascending=False)
print(missing_df)


                                     Missing Count  Missing Percent
last_contact_days_to                          2906        26.039427
tumor_status                                   697         6.245520
histological_type                              670         6.003584
DSS                                            512         4.587814
cause_of_death                                 366         3.279570
PFI.time                                       253         2.267025
DSS.time                                       252         2.258065
PFI                                            207         1.854839
initial_pathologic_dx_year                     130         1.164875
birth_days_to                                  119         1.066308
OS.time                                         66         0.591398
age_at_initial_pathologic_diagnosis             51         0.456989
treatment_outcome_first_course                  51         0.456989
OS                                              

Features such as overall survival (`OS`), progression-free interval (`PFI`), disease-specific survival (`DSS`), `vital_status`, `tumor_status`, `initial_pathologic_dx_year`, `birth_days_to` and `last_contact_days_to`,  might not be relevant for this use case. Also, `histological_type` is redundant with `type`, which is our intended label, while having more missing values, so we must remove it.

In [55]:
cdr_df = cdr_df.drop(columns=['Unnamed: 0', 'OS', 'PFI', 'DSS',
                              'OS.time', 'DSS.time', 'PFI.time',
                              'vital_status', 'tumor_status', 
                              'initial_pathologic_dx_year', 'birth_days_to',
                              'last_contact_days_to', 'histological_type'], axis=1)
cdr_df.head()

,type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_grade,menopause_status,cause_of_death,treatment_outcome_first_course
sample_id,,,,,,,,,,
TCGA-OR-A5J1,ACC,58.0,MALE,WHITE,Stage II,[Not Applicable],[Not Available],[Not Available],[Not Available],Complete Remission/Response
TCGA-OR-A5J2,ACC,44.0,FEMALE,WHITE,Stage IV,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease
TCGA-OR-A5J3,ACC,23.0,FEMALE,WHITE,Stage III,[Not Applicable],[Not Available],[Not Available],[Not Available],Complete Remission/Response
TCGA-OR-A5J4,ACC,23.0,FEMALE,WHITE,Stage IV,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease
TCGA-OR-A5J5,ACC,30.0,MALE,WHITE,Stage III,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease


Change label name to a more intuitive one:

In [56]:
cdr_df = cdr_df.rename(columns={'type': 'tumor_type_label'})
cdr_df.head()

,tumor_type_label,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_grade,menopause_status,cause_of_death,treatment_outcome_first_course
sample_id,,,,,,,,,,
TCGA-OR-A5J1,ACC,58.0,MALE,WHITE,Stage II,[Not Applicable],[Not Available],[Not Available],[Not Available],Complete Remission/Response
TCGA-OR-A5J2,ACC,44.0,FEMALE,WHITE,Stage IV,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease
TCGA-OR-A5J3,ACC,23.0,FEMALE,WHITE,Stage III,[Not Applicable],[Not Available],[Not Available],[Not Available],Complete Remission/Response
TCGA-OR-A5J4,ACC,23.0,FEMALE,WHITE,Stage IV,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease
TCGA-OR-A5J5,ACC,30.0,MALE,WHITE,Stage III,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease


### Converting categorical features to numeric

In [57]:
cdr_df.gender.value_counts()

gender
FEMALE    5815
MALE      5345
Name: count, dtype: int64

In [58]:
cdr_df.race.value_counts()

race
WHITE                                        8186
[Not Available]                              1038
BLACK OR AFRICAN AMERICAN                     934
ASIAN                                         675
[Not Evaluated]                               155
[Unknown]                                     132
AMERICAN INDIAN OR ALASKA NATIVE               27
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER      13
Name: count, dtype: int64

In [59]:
cdr_df.ajcc_pathologic_tumor_stage.value_counts()

ajcc_pathologic_tumor_stage
[Not Available]     2280
[Not Applicable]    1864
Stage I             1286
Stage IIA            804
Stage III            709
Stage IIB            702
Stage II             642
Stage IIIA           507
Stage IV             479
Stage IB             401
Stage IA             383
Stage IVA            336
Stage IIIB           286
Stage IIIC           248
Stage IIC             67
[Discrepancy]         53
IS                    46
Stage IVB             21
I/II NOS              14
Stage X               13
Stage IVC              7
Stage 0                7
[Unknown]              5
Name: count, dtype: int64

Encode gender:

In [60]:
cdr_df.gender = cdr_df.gender.apply(lambda x: 1 if x.lower() == 'male' else 0)
cdr_df.gender.value_counts()

gender
0    5815
1    5345
Name: count, dtype: int64

Encode race and tumor stage:

In [61]:
features_to_encode = ['race', 'ajcc_pathologic_tumor_stage']

In [62]:
# Dictionary that will contain the mapping between the categories and their encodings
encod_dict = dict([('gender', dict([('male' , 1), ('female', 0)]))])

In [66]:
encod_dict = {}

for feature in features_to_encode:
    # Factorize returns encoded values and unique categories
    codes, uniques = pandas.factorize(cdr_df[feature])
    cdr_df[feature] = codes
    encod_dict[feature] = dict(enumerate(uniques))


In [67]:
cdr_df.head()

,tumor_type_label,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_grade,menopause_status,cause_of_death,treatment_outcome_first_course
sample_id,,,,,,,,,,
TCGA-OR-A5J1,ACC,58.0,1,0,0,[Not Applicable],[Not Available],[Not Available],[Not Available],Complete Remission/Response
TCGA-OR-A5J2,ACC,44.0,0,0,1,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease
TCGA-OR-A5J3,ACC,23.0,0,0,2,[Not Applicable],[Not Available],[Not Available],[Not Available],Complete Remission/Response
TCGA-OR-A5J4,ACC,23.0,0,0,1,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease
TCGA-OR-A5J5,ACC,30.0,1,0,2,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease


In [68]:
encod_dict

{'race': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7},
 'ajcc_pathologic_tumor_stage': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22}}

### Normalizing data

In this table, we only need to normalize the age.

In [69]:
cdr_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age_at_initial_pathologic_diagnosis,11109.0,59.099019,14.415440,10.0,50.0,60.0,70.0,90.0
gender,11160.0,0.478943,0.499579,0.0,0.0,0.0,1.0,1.0
race,11160.0,0.827957,1.630037,0.0,0.0,0.0,1.0,7.0
ajcc_pathologic_tumor_stage,11160.0,7.428226,5.716661,0.0,3.0,6.0,11.0,22.0


Save the dataframe before normalizing:

In [70]:
# Ensure the directory exists
unnorm_dir = os.path.join(data_path, 'cleaned/unnormalized')
os.makedirs(unnorm_dir, exist_ok=True)

# Now save the file
cdr_df.to_csv(os.path.join(unnorm_dir, 'clinical_outcome.csv'), index=False)


Normalize the data into a new dataframe:

In [71]:
scaler = StandardScaler()

# Copy original dataframe
cdr_df_norm = cdr_df.copy()

# Normalize the specified column
column = 'age_at_initial_pathologic_diagnosis'
cdr_df_norm[column] = scaler.fit_transform(cdr_df[[column]])

cdr_df_norm.head()


,tumor_type_label,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_grade,menopause_status,cause_of_death,treatment_outcome_first_course
sample_id,,,,,,,,,,
TCGA-OR-A5J1,ACC,-0.076242,1,0,0,[Not Applicable],[Not Available],[Not Available],[Not Available],Complete Remission/Response
TCGA-OR-A5J2,ACC,-1.047467,0,0,1,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease
TCGA-OR-A5J3,ACC,-2.504304,0,0,2,[Not Applicable],[Not Available],[Not Available],[Not Available],Complete Remission/Response
TCGA-OR-A5J4,ACC,-2.504304,0,0,1,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease
TCGA-OR-A5J5,ACC,-2.018692,1,0,2,[Not Applicable],[Not Available],[Not Available],[Not Available],Progressive Disease


Confirm that everything is ok through the `describe` method:

In [72]:
cdr_df_norm.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age_at_initial_pathologic_diagnosis,11109.0,-1.714155e-16,1.000045,-3.406155,-0.631228,0.062504,0.756236,2.143699
gender,11160.0,4.789427e-01,0.499579,0.000000,0.000000,0.000000,1.000000,1.000000
race,11160.0,8.279570e-01,1.630037,0.000000,0.000000,0.000000,1.000000,7.000000
ajcc_pathologic_tumor_stage,11160.0,7.428226e+00,5.716661,0.000000,3.000000,6.000000,11.000000,22.000000


Save the normalized dataframe:

In [73]:
cdr_df_norm.to_csv(f'{data_path}cleaned/normalized/clinical_outcome.csv')

## Saving enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [74]:
stream = open(f'{data_path}cleaned/encod_dict.yaml', 'w')
yaml.dump(encod_dict, stream, default_flow_style=False)